In [1]:
#### Jupyter Basic Setting ####
import pandas as pd,numpy as np
import matplotlib.pyplot as plt,seaborn as sns  # 시각화
import warnings; warnings.filterwarnings('ignore')  # 경고 무시
import sys,os 
import xgboost as xgb
# xgb.set_config(verbosity=0)# xgb 경고 끔
pd.set_option('display.float_format', lambda x: '%.3f' % x) ##scientific 수치
parent_dir = os.path.dirname(os.getcwd())
parent_dir = os.path.dirname(parent_dir)
sys.path.append(parent_dir)
from Module.Functions import Service
# 기본 세팅
Service.plotSetting("seaborn-v0_8")


___ 🟡 PLOTSETTING. 
______ 📌 matplotlibn plot 한글화 Setting
___## OS platform 한글 세팅완료 ## ___


In [2]:
data_Folder='../../Data/승하차정제'
finalT =pd.read_csv(os.path.join(data_Folder,'finalTable_18to24.csv'))

In [3]:
test=finalT

In [5]:
test.head()

,역사코드,역이름,호선,수송일자,승하차구분,05시인원,06시인원,07시인원,08시인원,09시인원,...,15배차,16배차,17배차,18배차,19배차,20배차,21배차,22배차,23배차,24배차
0,150,서울역,8,2023-11-01,승차,342,520,1929,2864,2321,...,15.0,15.5,17.0,16.0,15.5,14.5,14.0,11.0,9.0,1.0
1,150,서울역,8,2023-11-01,하차,300,2166,5186,10500,6808,...,15.0,15.5,17.0,16.0,15.5,14.5,14.0,11.0,9.0,1.0
2,150,서울역,8,2023-11-02,승차,369,562,1859,2711,2522,...,15.0,15.5,17.0,16.0,15.5,14.5,14.0,11.0,9.0,1.0
3,150,서울역,8,2023-11-02,하차,361,2284,5117,10856,7006,...,15.0,15.5,17.0,16.0,15.5,14.5,14.0,11.0,9.0,1.0
4,150,서울역,8,2023-11-03,승차,340,513,1681,2821,2419,...,15.0,15.5,17.0,16.0,15.5,14.5,14.0,11.0,9.0,1.0


In [80]:
test.describe()
Service.dataInfoProcessing(test,replace_Nan=True)
test.to_csv(os.path.join(data_Folder,'finalTable_18to24.csv'))

___ 🟡 DATAINFOPROCESSING. 
______ 📌 Data frame 의 정제해야할 부분을 체크해주는 함수 입니다
  1️⃣ Data row/colum numbers : 3958770/51
  2️⃣ null check 결과Null 없는 clean data!
  3️⃣ Column  Information (중복체크)
	idx.columName |				 |Colum Info(dtype)|** 
	 ----------------------------------------------------------------
	0.[역사코드(int64)]: 			283/3958770 [uniq/raw]
	1.[역이름(object)]: 			253/3958770 [uniq/raw]
	2.[호선(int64)]: 			1/3958770 [uniq/raw]
	3.[수송일자(object)]: 			2222/3958770 [uniq/raw]
	4.[승하차구분(object)]: 			2/3958770 [uniq/raw]
	5.[05시인원(int64)]: 			2134/3958770 [uniq/raw]
	 ...etc (추가로 45개의 칼럼이 있습니다 )


### 데이터 분석
> 

In [11]:
test_2024=test[test['년도']==2024]

In [14]:
test_st=test_2024[['역사코드', '역이름', '호선', '수송일자', '승하차구분', '05시인원', '06시인원', '07시인원',
       '08시인원', '09시인원', '10시인원', '11시인원', '12시인원', '13시인원', '14시인원', '15시인원',
       '16시인원', '17시인원', '18시인원', '19시인원', '20시인원', '21시인원', '22시인원', '23시인원',
       '24시인원', '요일', '주중주말', '년도', '월', '주차', '공휴일']]

In [22]:
test_2024.columns

Index(['역사코드', '역이름', '호선', '수송일자', '승하차구분', '05시인원', '06시인원', '07시인원',
       '08시인원', '09시인원', '10시인원', '11시인원', '12시인원', '13시인원', '14시인원', '15시인원',
       '16시인원', '17시인원', '18시인원', '19시인원', '20시인원', '21시인원', '22시인원', '23시인원',
       '24시인원', '요일', '주중주말', '년도', '월', '주차', '공휴일', '05배차', '06배차', '07배차',
       '08배차', '09배차', '10배차', '11배차', '12배차', '13배차', '14배차', '15배차', '16배차',
       '17배차', '18배차', '19배차', '20배차', '21배차', '22배차', '23배차', '24배차'],
      dtype='object')

---
### 회귀 모델 만들기
- input 과 output 을 나눈다

---
### Test, trainning 나누기 

In [74]:
def linearRegression_test(data):
    # linear regression 을 위해 수치화할수있는것만 남긴다. 
    test_2024_승차 =data[data['승하차구분']=='승차']
    test_2024_하차 =data[data['승하차구분']=='하차']
    X= test_2024_승차[['역사코드', '호선','요일', '년도', '월', '주차', '공휴일','05배차', '06배차', '07배차',
        '08배차', '09배차', '10배차', '11배차', '12배차', '13배차', '14배차', '15배차', '16배차',
        '17배차', '18배차', '19배차', '20배차', '21배차', '22배차', '23배차', '24배차']]
    y =test_2024_승차[['05시인원', '06시인원', '07시인원',
        '08시인원', '09시인원', '10시인원', '11시인원', '12시인원', '13시인원', '14시인원', '15시인원',
        '16시인원', '17시인원', '18시인원', '19시인원', '20시인원', '21시인원', '22시인원', '23시인원',
        '24시인원']]
    ## 승하차구분, 주중주말 삭제 

    from sklearn.model_selection import train_test_split
    from sklearn.linear_model import LinearRegression

    X_train,X_test, y_train,y_test = train_test_split(X,y,random_state=2)
    lin_reg = LinearRegression()
    # lin_reg.fit(X_train,y_train.iloc[:,1])
    lin_reg.fit(X_train,y_train)
    y_pred = lin_reg.predict(X_test)
    
    from sklearn.metrics import mean_squared_error
    mse = mean_squared_error(y_test,y_pred)
    rmse = np.sqrt(mse)
    print(f"RMSE: {rmse :0.2f}")
    return y_pred

# test_2024=test[test['년도']==2022]
# test_2024=test[test['년도']==2024]
linearRegression_test(test)

RMSE: 858.03


array([[ 349.59897683,  630.25212832, 1736.18307186, ...,  649.9631509 ,
         293.09105426,   31.70789856],
       [ 188.47344188,  296.6328327 ,  816.8868605 , ...,  339.60694319,
         149.42088106,   52.70366457],
       [ 176.62702506,  320.72598097,  906.50050132, ...,  389.33025098,
         179.3450799 ,   47.59625311],
       ...,
       [ 212.49960047,  358.80257931,  945.2491974 , ...,  595.50224698,
         279.13410475,   45.82470456],
       [ 106.71931897,  207.97023716,  508.61907326, ...,  154.58048078,
          85.47755882, -110.27324941],
       [ 179.26370855,  267.33437077,  470.32298841, ...,  431.4285946 ,
         181.57233351,   -7.09972853]])

---
### Linear Regression

In [79]:
# linear regression 을 위해 수치화할수있는것만 남긴다. 
test_승차 =test[test['승하차구분']=='승차']
test_하차 =test[test['승하차구분']=='하차']
X= test_승차[['역사코드', '호선','요일', '년도', '월', '주차', '공휴일','05배차', '06배차', '07배차',
    '08배차', '09배차', '10배차', '11배차', '12배차', '13배차', '14배차', '15배차', '16배차',
    '17배차', '18배차', '19배차', '20배차', '21배차', '22배차', '23배차', '24배차']]
y =test_승차[['05시인원', '06시인원', '07시인원',
    '08시인원', '09시인원', '10시인원', '11시인원', '12시인원', '13시인원', '14시인원', '15시인원',
    '16시인원', '17시인원', '18시인원', '19시인원', '20시인원', '21시인원', '22시인원', '23시인원',
    '24시인원']]
## 승하차구분, 주중주말 삭제 

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

X_train,X_test, y_train,y_test = train_test_split(X,y,random_state=2)
lin_reg = LinearRegression()
# lin_reg.fit(X_train,y_train.iloc[:,1])
lin_reg.fit(X_train,y_train)
y_pred = lin_reg.predict(X_test)

from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test,y_pred)
rmse = np.sqrt(mse)
print(f"RMSE: {rmse :0.2f}")

RMSE: 858.03


In [85]:
test.tail(3)

,역사코드,역이름,호선,수송일자,승하차구분,05시인원,06시인원,07시인원,08시인원,09시인원,...,15배차,16배차,17배차,18배차,19배차,20배차,21배차,22배차,23배차,24배차
3958767,2761,부평구청,8,2018-12-30,하차,8,79,95,94,131,...,5.000,5.000,5.500,8.500,7.500,8.000,5.000,5.000,5.000,1.000
3958768,2761,부평구청,8,2018-12-31,승차,192,650,1995,1605,768,...,5.000,5.000,5.500,8.500,7.500,8.000,5.000,5.000,5.000,1.000
3958769,2761,부평구청,8,2018-12-31,하차,30,168,267,457,227,...,5.000,5.000,5.500,8.500,7.500,8.000,5.000,5.000,5.000,1.000


In [49]:
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test,y_pred)
rmse = np.sqrt(mse)
print(f"RMSE: {rmse :0.2f}")

RMSE: 871.71


---
### RMSE 871 값은  좋은값인가?

In [84]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
test_승차['09시인원'].describe()

count   1979385.000
mean        843.479
std         826.086
min           0.000
25%         337.000
50%         605.000
75%        1059.000
max        9797.000
Name: 09시인원, dtype: float64

---
### xGBoost model regression

In [87]:
from xgboost import XGBRegressor
xg_reg  = XGBRegressor()
xg_reg.fit(X_train,y_train)
y_pred  = xg_reg.predict(X_test)
mse = mean_squared_error(y_test,y_pred)
rmse = np.sqrt(mse)
print(f"RMSE: {rmse :0.2f}")


In [88]:
y_pred  = xg_reg.predict(X_test)
mse = mean_squared_error(y_test,y_pred)
rmse = np.sqrt(mse)
print(f"RMSE: {rmse :0.2f}")

RMSE: 181.64


---
### Cross Validation

